In [1]:
import os

import numpy as np
import pandas as pd

from typing import List
!pip install wandb
import wandb

import sklearn
import logging
!pip install simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.1/281.1 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2

In [2]:
!export WANDB_API_KEY="09d02010ef9c7078a54f49356384e4d19d8f2e81"

In [3]:
# sweep_config = {
#     'method': 'random',  # Other methods include "grid", "bayesian", "random"
#     'metric': {
#         'name': 'eval_loss',
#         'goal': 'minimize'
#     },
#     'parameters': {
#         'learning_rate': {
#             'min': 5e-6,
#             'max': 5e-5
#         },
#         'weight_decay': {
#           'values': [0, 5e-4, 1e-3]
#         },
#         'train_batch_size': {
#             'values': [16, 32, 64]
#         }
#     }
# }

sweep_config = {
    'method': 'random',  # Other methods include "grid", "bayesian", "random"
    'metric': {
        'name': 'eval_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'values': [4e-5]
        },
        'weight_decay': {
          'values': [0]
        },
        'train_batch_size': {
            'values': [8]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project='chemberta2-ctFDA-hparams')

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: sh7nkxia
Sweep URL: https://wandb.ai/data-fusion/chemberta2-ctFDA-hparams/sweeps/sh7nkxia


In [4]:
train_df = pd.read_csv("train_clintoxfda_1.csv")
valid_df = pd.read_csv("valid_clintoxfda_1.csv")

# set the logging directories
project_name = "chemberta-2-clintxfda-stf"
output_path = '/home/yutonglu/projects/def-hup-ab/yutonglu/chemberta-2/'
model_name = "77m-MTR"

model_folder = os.path.join(output_path, model_name)

evaluation_folder = os.path.join(output_path, model_name + '_evaluation')
if not os.path.exists(evaluation_folder):
    os.makedirs(evaluation_folder)

wandb.login()

# configure training
model_args = ClassificationArgs()
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.max_seq_length = 512
model_args.num_train_epochs = 100
model_args.manual_seed = 4
model_args.use_multiprocessing = True
model_args.evaluate_each_epoch = True
model_args.evaluate_during_training_verbose = True
model_args.evaluate_during_training = True
# model_args.use_early_stopping = True
# model_args.optimizer = 'AdamW'
# model_args.early_stopping_patience = 10,
# model_args.early_stopping_delta = 0.001,
# model_args.early_stopping_metrics = 'eval_loss'
# model_args.early_stopping_metrics_minimize = True
# model_args.early_stopping_consider_epochs =  True
model_args.auto_weights = True

wandb: Currently logged in as: yutong-lu (data-fusion). Use `wandb login --relogin` to force relogin


In [5]:
def train():
    # Initialize a new wandb run
    wandb.init()

    # Create a TransformerModel
    model = ClassificationModel(
        "roberta",
        'DeepChem/ChemBERTa-77M-MLM',
        args=model_args,
        sweep_config=wandb.config,
    )

    # Train the model
    model.train_model(train_df, eval_df = valid_df)

    # Evaluate the model
    # model.eval_model(valid_df)

    # Sync wandb
    wandb.join()

wandb.agent(sweep_id, train, count = 1)

wandb: Agent Starting Run: 7som4mew with config:
wandb: 	learning_rate: 4e-05
wandb: 	train_batch_size: 8
wandb: 	weight_decay: 0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/13.7M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

0it [00:00, ?it/s]

Running Epoch 100 of 100:   0%|          | 0/112 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


0it [00:00, ?it/s]

Training loss,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▄▄▄▄▄▄▄▁▄▄▁█▁▄▄▄▄▄▁▁▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
auprc,▁▄██████████████████████████████████████
auroc,▁▃██████████████████████████████████████
eval_loss,█▂▁▁▁▂▂▂▂▂▂▁▂▂▂▂▂▂▃▂▃▂▂▂▂▂▃▃▃▃▃▃▂▂▃▃▃▃▃▃
f1_score,▅▅▅▅▅▅▅▁▅▅▁█▁▅▅▅▅▅▁▁▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
fn,▁▁▆▆▆▆▆█▆▆█▃█▆▆▆▆▆██▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
fp,██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr,▂▄▇███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
mcc,▁▁▇▇▇▇▇▆▇▇▆█▆▇▇▇▇▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
